In [1]:
import torch
import torch.nn as nn
import torch.distributed as dist


import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm

from datasets import load_dataset, load_from_disk
from transformers import (
    AutoConfig, AutoTokenizer, DataCollatorForLanguageModeling
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from torch.utils.data import DataLoader
from torch.nn import CrossEntropyLoss

# API KEY를 환경변수로 관리하기 위한 설정 파일
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
from sklearn.metrics import accuracy_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score


/home/woogoonkyu/anaconda3/envs/rapa_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]= "1"


In [3]:
from vllm import LLM, SamplingParams
import pandas as pd
import torch
import torch._dynamo



# torch._dynamo.config.suppress_errors = True

INFO 03-28 01:57:34 [__init__.py:239] Automatically detected platform cuda.


2025-03-28 01:57:34,457	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [ ]:
from datasets import load_from_disk, DatasetDict, Dataset

dataset = load_from_disk("cleaned_dataset_with_answer_tag_new") # Preprocessing에서 저장한 데이터 셋

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'source'],
        num_rows: 15475
    })
    test: Dataset({
        features: ['input', 'output', 'source'],
        num_rows: 3870
    })
})

In [6]:
print(dataset['test']['input'][0])

You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
아니요, 하지 않아요.
이 증상은 위식도역류병(GERD)일 수 있어요. 목구멍에 음식이 걸리는 것은 식도 협착 같은 합병증이 있을 수도 있어요. 즉각적으로는 프로톤 펌프 억제제(PPI)와 생활습관 개선을 시도해보는 것이 좋아요. 하지만 목구멍에 음식이 걸리는 증상이 있으므로 식도 내시경 검사를 해야 합니다. 이를 통해 식도염, 협착, 바렛식도 등의 합병증 여부를 확인할 수 있습니다.
그럼 어떻게 대처해야 할까요?

***Answer:


In [7]:
question = []
y_true = []
for input, source, output in zip(dataset['test']['input'], dataset['test']['source'], dataset['test']['output']):
    if source == 'dialogue':
        question.append(input)
        y_true.append(output)
    

In [8]:
print(question[1])

You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
그 외에는 아무런 증상이 없어요.
좋아요, 이해했습니다. 가슴 증상으로 인한 심장 질환은 걱정하지 않아도 됩니다. 심전도 검사 결과가 정상이라면요. 그런데 불안과 우울증으로 고생하고 계시다면, 가슴 증상은 불안이 제어되지 못한 것으로 보입니다. 정신과 전문의를 찾아 상담을 받는 것이 좋겠어요. 삶에서 스트레스 요인을 찾아 그 해결책을 찾아보세요. 안정제와 항우울제가 필요할 수도 있습니다. 걱정하지 마세요, 남편께서는 괜찮아질 거에요. 문제 해결에 도움이 되었나요?
네, 감사합니다. 어떻게 대처해야 할지 궁금합니다.

***Answer:


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList, LogitsProcessor
from transformers import LogitsProcessor
import re


## Blockin English, Chinese, Japanese + 특수문자 Unicdoe
class ForeignLangWithEnglishOnceBlocker(LogitsProcessor):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.foreign_lang_mask = None
        self.english_seen = False  # 영어가 한 번이라도 등장했는지 추적

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor) -> torch.FloatTensor:
        device = scores.device

        # 토큰 전체 디코딩 → 영어가 나왔는지 확인
        decoded_so_far = self.tokenizer.batch_decode(input_ids, skip_special_tokens=True)
        full_text = " ".join(decoded_so_far)

        if not self.english_seen and re.search(r'[a-zA-Z]', full_text):
            self.english_seen = True  # 영어가 처음 등장함

        # foreign_lang_mask 처음 초기화
        if self.foreign_lang_mask is None:
            token_ids = torch.arange(scores.size(-1), device=device)
            decoded_tokens = self.tokenizer.batch_decode(token_ids.unsqueeze(1), skip_special_tokens=True)

            # Unicode 범위 정의
            chinese_ranges = [(0x4E00, 0x9FFF), (0x3400, 0x4DBF), (0x20000, 0x2A6DF), (0xF900, 0xFAFF)]
            japanese_ranges = [(0x3040, 0x309F), (0x30A0, 0x30FF), (0x31F0, 0x31FF)]
            russian_ranges = [(0x0400, 0x04FF), (0x0500, 0x052F)]
            special_char_ranges = [(0x2000, 0x206F),  # 일반 특수문자 (예: ‘ ” …)
                                   (0x2190, 0x21FF),  # 화살표
                                   (0x2300, 0x23FF),  # 기술 기호
                                   (0x2500, 0x257F),  # 박스 드로잉
                                   (0x25A0, 0x25FF),  # 기호
                                   (0x2B00, 0x2BFF),  # 기타 기호
                                   (0x1F000, 0x1F9FF)]  # 이모지 등

            all_ranges = chinese_ranges + japanese_ranges + special_char_ranges

            def is_foreign(token):
                return any(
                    any(start <= ord(c) <= end for start, end in all_ranges)
                    for c in token if c.strip()
                )

            def is_english(token):
                return any(c.isascii() and c.isalpha() for c in token)

            # 두 개의 마스크 생성
            self.foreign_lang_mask = torch.tensor([is_foreign(t) for t in decoded_tokens], device=device)
            self.english_mask = torch.tensor([is_english(t) for t in decoded_tokens], device=device)

        # 무조건 차단할 외국어 (중국어, 일본어, 러시아어)
        scores[:, self.foreign_lang_mask] = -float("inf")

        # 영어는 아직 등장 안 했으면 차단
        if not self.english_seen:
            scores[:, self.english_mask] = -float("inf")

        return scores




In [ ]:

# 1. 토크나이저 및 모델 로드
model_name = "RAPA_model_name" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

Loading checkpoint shards: 100%|██████████| 7/7 [00:03<00:00,  2.31it/s]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [ ]:
# 배치 추론 함수
batch_size=2
max_new_tokens=128
all_outputs = []

In [ ]:
from transformers import LogitsProcessorList

def make_custom_logits_filter(tokenizer):
    """
    ForeignLangWithEnglishOnceBlocker를 logits processor 리스트로 감싸서 반환하는 함수.
    """
    processors = LogitsProcessorList()
    processors.append(ForeignLangWithEnglishOnceBlocker(tokenizer))
    return processors


In [ ]:
# 배치 추론 함수
batch_size=2
max_new_tokens=128
all_outputs = []
for i in tqdm(range(0, len(question), batch_size)):
    
    batch = question[i:i+batch_size]
    inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).to("cuda")

    logits_processor = LogitsProcessorList()
    logits_processor.append(make_custom_logits_filter(tokenizer))

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            logits_processor=logits_processor,
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    all_outputs.extend(decoded)


  0%|          | 0/737 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  0%|          | 1/737 [00:19<3:59:16, 19.51s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  0%|          | 2/737 [00:38<3:54:40, 19.16s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
  0%|          | 3/737 [00:57<3:51:58, 18.96s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
A d

In [ ]:
# pd.DataFrame(all_outputs).to_csv('임시.csv', encoding = 'utf-8-sig')

In [83]:
all_outputs = pd.read_csv('임시.csv').iloc[:,1].tolist()

In [84]:
all_outputs

['You are a medical professional proficient in both Korean and English.\nYour task is to carefully read the conversation between a Client and a Doctor, \nthen predict the next response from the Doctor in a professional and empathetic manner.\n\nPlease respond in **Korean** only.\n\nNow produce the next Doctor response to the Client’s last question.\nReturn your answer in Korean.\n\nYou will always respond as the doctor, and your answer must be written in Korean only.\n\n***Conversation:\n아니요, 하지 않아요.\n이 증상은 위식도역류병(GERD)일 수 있어요. 목구멍에 음식이 걸리는 것은 식도 협착 같은 합병증이 있을 수도 있어요. 즉각적으로는 프로톤 펌프 억제제(PPI)와 생활습관 개선을 시도해보는 것이 좋아요. 하지만 목구멍에 음식이 걸리는 증상이 있으므로 식도 내시경 검사를 해야 합니다. 이를 통해 식도염, 협착, 바렛식도 등의 합병증 여부를 확인할 수 있습니다.\n그럼 어떻게 대처해야 할까요?\n\n***Answer:-식도 내시경 검사를 통해 정확한 진단을 받은 후에 PPI를 사용하여 증상을 완화시킬 수 있습니다. 또한 생활습관 개선, 예를 들어 식사 후에 바로 눕지 않기, 음식물 섭취를 조절하는 등의 방법으로 증상을 관리할 수 있습니다. -식도염, 협착, 바렛식도 등의 합병증이 확인되면 그에 따라 적절한 치료를 받을 수 있습니다. -식도 내시경 검사 결과를 통해 정확한 진단을 받은 후에',
 'You are a medical professional proficient i

In [85]:
len(all_outputs), len(question)

(2371, 2372)

# 질문 누락 확인

In [86]:
mismatched_question_indices = []

for i in range(len(all_outputs)):
    out = all_outputs[i]
    q_curr = question[i]
    
    if len(out) >= len(q_curr) and out[:len(q_curr)] == q_curr:
        continue  # 정상 매칭

    if i + 1 < len(question):
        q_next = question[i + 1]
        if len(out) >= len(q_next) and out[:len(q_next)] == q_next:
            continue  # 다음 질문과 매칭됨

    # 매칭 실패 → question[i]가 매칭 안 됐다고 간주
    mismatched_question_indices.append(i)

print(f"총 mismatched question 개수: {len(mismatched_question_indices)}")
print(f"Mismatch된 질문 인덱스 예시: {mismatched_question_indices[:10]}")


총 mismatched question 개수: 1
Mismatch된 질문 인덱스 예시: [1975]


In [ ]:
index_to_redo = 1975
q = question[index_to_redo]

inputs = tokenizer([q], return_tensors="pt", padding=True, truncation=True).to("cuda")

logits_processor = LogitsProcessorList()
logits_processor.append(make_custom_logits_filter(tokenizer))

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        logits_processor=logits_processor,
    )

decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
new_output = decoded[0]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
all_outputs.insert(1974, new_output)


In [87]:
# 마지막 질문이 마지막 출력에 포함됐는지 확인
last_q = question[-1]
last_out = all_outputs[-1]

if len(last_out) < len(last_q) or last_out[:len(last_q)] != last_q:
    print("⚠️ 마지막 질문도 출력에 포함되지 않았음!")
    print("추가로 하나 더 누락된 질문이 존재합니다.")
    print(f"질문 인덱스: {len(question) - 1}")
    print(f"질문 내용: {last_q[:100]}...")
else:
    print("✅ 마지막 질문은 정상적으로 포함되어 있음.")


✅ 마지막 질문은 정상적으로 포함되어 있음.


In [92]:
# 누락된 질문 인덱스
missing_index = 1975

# 새로운 output을 해당 위치에 삽입하되,
# 그 이전까지는 그대로, 이후는 원래보다 하나씩 뒤로 shift해서 매칭되도록 함
corrected_outputs = (
    all_outputs[:missing_index] +       # 앞부분은 문제 없음
    [new_output] +                      # 누락된 답변 추가
    all_outputs[missing_index:]         # 뒤는 원래보다 하나씩 앞으로 당겨서 매칭됨
)

In [93]:
len(corrected_outputs), len(question)

(2372, 2372)

In [99]:
len(corrected_outputs), len(question)

(2372, 2372)

In [97]:
from transformers import LogitsProcessorList

bad_index = 1976
q = question[bad_index]

# 토크나이징
inputs = tokenizer([q], return_tensors="pt", padding=True, truncation=True).to("cuda")

# logits processor
logits_processor = LogitsProcessorList()
logits_processor.append(make_custom_logits_filter(tokenizer))

# 모델 추론
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False,
        logits_processor=logits_processor,
    )

# 디코딩
decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
corrected_outputs[bad_index] = decoded[0]  # 대체!

print(f"✅ index {bad_index}의 출력 재생성 완료 및 덮어쓰기 완료!")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


✅ index 1976의 출력 재생성 완료 및 덮어쓰기 완료!


In [98]:
mismatched_question_indices = []

for i in range(len(corrected_outputs)):
    out = corrected_outputs[i]
    q_curr = question[i]
    
    if len(out) >= len(q_curr) and out[:len(q_curr)] == q_curr:
        continue

    if i + 1 < len(question):
        q_next = question[i + 1]
        if len(out) >= len(q_next) and out[:len(q_next)] == q_next:
            continue

    mismatched_question_indices.append(i)

print(f"총 mismatched question 개수: {len(mismatched_question_indices)}")
print(f"Mismatch된 질문 인덱스 예시: {mismatched_question_indices[:10]}")


총 mismatched question 개수: 1
Mismatch된 질문 인덱스 예시: [1976]


In [39]:
# y_pred_real = [i.outputs[0].text for i in y_pred]/

In [40]:
# # Sampling parameters 설정
# sampling_params = SamplingParams(
#     temperature=0.0,
#     max_tokens=128,
#     logits_processors=[processor]

# )

# llm = LLM(
#     model=model_name,
#     # tensor_parallel_size=1,  # 안정성 위해 GPU 1개만 사용
# )

# #/nas_homes/seunguk/hf_llm/models--saltlux--Ko-Llama3-Luxia-8B/snapshots/14533c55dc476c57220427bc65ce2539133b37db
# # /nas_homes/seunguk/hf_llm/models--beomi--Llama-3-KoEn-8B-Instruct-preview/snapshots/c9f8288972cc11d27fff2062d2412891d8354938
# # /nas_homes/seunguk/hf_llm/models--beomi--Llama-3-Open-Ko-8B/snapshots/a8e8214f79f0d2cea817020a93dca48e8a6be18b
# # /nas_homes/seunguk/hf_llm/models--MLP-KTLim--llama-3-Korean-Bllossom-8B/snapshots/ed9647c18477ee09a03690c613c859eddca24362
# # /home/woogoonkyu/RAPA_inference/Additional_model_2/openbiollm_chatvector_new

# # /nas_homes/seunguk/hf_llm/models--LGAI-EXAONE--EXAONE-3.5-7.8B-Instruct/snapshots/0ff6b5ec7c13b049b253a16a889aa269e6b79a94
# # /nas_homes/seunguk/hf_llm/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796
# # /nas_homes/projects/rapa/merged_llama3_openbio_new_chatvector_adalora
# # beomi/Llama-3-KoEn-8B-Instruct-preview + chatvector + AdaLoRA + Block 적용
# # /nas_homes/seunguk/hf_llm/models--beomi--Llama-3-KoEn-8B/snapshots/43bacd328630f27df0438cabd8c1baa7058c4769


# print("before")
# # vLLM을 사용한 병렬 추론
# outputs = llm.generate(question, sampling_params)

# print("after")

# # 정답 및 예측값 저장
# y_pred = outputs


In [20]:
y_pred_real = all_outputs

In [25]:
y_true_real =  y_true

In [28]:
df_blocking = pd.DataFrame([y_pred_real, y_true_real]).T
df_blocking.columns = ['Blocking', 'real']

In [30]:
df_blocking.to_csv('Blocking.csv', encoding = 'utf-8-sig')

In [24]:
y_pred_real[0]

'You are a medical professional proficient in both Korean and English.\nYour task is to carefully read the conversation between a Client and a Doctor, \nthen predict the next response from the Doctor in a professional and empathetic manner.\n\nPlease respond in **Korean** only.\n\nNow produce the next Doctor response to the Client’s last question.\nReturn your answer in Korean.\n\nYou will always respond as the doctor, and your answer must be written in Korean only.\n\n***Conversation:\n아니요, 하지 않아요.\n이 증상은 위식도역류병(GERD)일 수 있어요. 목구멍에 음식이 걸리는 것은 식도 협착 같은 합병증이 있을 수도 있어요. 즉각적으로는 프로톤 펌프 억제제(PPI)와 생활습관 개선을 시도해보는 것이 좋아요. 하지만 목구멍에 음식이 걸리는 증상이 있으므로 식도 내시경 검사를 해야 합니다. 이를 통해 식도염, 협착, 바렛식도 등의 합병증 여부를 확인할 수 있습니다.\n그럼 어떻게 대처해야 할까요?\n\n***Answer:-식도 내시경 검사를 통해 정확한 진단을 받은 후에 PPI를 사용하여 증상을 완화시킬 수 있습니다. 또한 생활습관 개선, 예를 들어 식사 후에 바로 눕지 않기, 음식물 섭취를 조절하는 등의 방법으로 증상을 관리할 수 있습니다. -식도염, 협착, 바렛식도 등의 합병증이 확인되면 그에 따라 적절한 치료를 받을 수 있습니다. -식도 내시경 검사 결과를 통해 정확한 진단을 받은 후에'

In [ ]:
y_pred_real_real = [i for i in y_pred_real]

In [37]:
print(y_pred_real[-1])

You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
저는 56세 여성인데 갑자기 복통이 심하고 구토하고, 저혈압과 맥박이 빨라져서 쇼크 증상이 있어요. 가끔 가벼운 왼쪽 복통이 있던 적이 있어요.
왼쪽 복통은 언제부터 있었나요?
몇 달 전부터 가끔씩 있었어요.

***Answer: (의사) 환자분의 증상으로 보아 급성 복통과 구토, 저혈압, 빠른 맥박으로 인한 쇼크 증상이 나타나고 있습니다. 이에 대한 정확한 진단을 위해 복부 CT 촬영을 추천합니다. CT 촬영을 통해 환자분의 복부 내부를 자세히 확인하여 원인을 파악할 수 있습니다. CT 촬영 후 결과를 토대로 적절한 치료를 진행할 수 있습니다. CT 촬영에 대한 동의를 구합니다. (의사) *** (의


In [38]:
print(y_true[-1])

복통은 어떤 형태였나요? 급격한 변화가 있었나요?


In [ ]:
y_pred_real = [i.outputs[0].text for i in y_pred_real_real]

# 결과 출력
for i, (true_ans, pred_ans) in enumerate(zip(y_true, y_pred_real)):
    print(f"Q{i+1}: True: {true_ans}, / {pred_ans}")

Q1: True: 식도 내시경 검사를 통해 합병증을 확인하고 치료 계획을 세울 수 있을 거예요. 추가적으로 식습관과 생활습관을 개선하는 것이 중요합니다. 음식을 덜 씹거나 빨리 먹지 않도록 주의하고, 식사 후에는 바로 누워서는 안 됩니다. 또한, 과식이나 과음을 피하고, 알코올과 카페인을 제한하는 것도 도움이 될 수 있어요. 이러한 조치들을 함께 시도해 보시고, 내시경 검사 결과에 따라 추가적인 치료 방법을 결정하도록 해요., / 
Q2: True: 처방전을 받기 위해 정신과 전문의를 찾아보세요. 그리고 상담 세션을 시작해보세요. 스트레스 요인을 찾아내고 그 해결책을 찾는 데에도 도움을 받을 수 있을 거예요., / 
Q3: True: 이러한 증상은 눈에 심각한 손상을 초래할 수 있으므로, 병원에서 수술을 해야 합니다. 또한, 항생제를 투여하여 감염을 예방해야 합니다., /  안녕하세요. 안전한 시술을 위해 먼저 눈을 청결하게 해야 합니다. 이물질이 눈에 박혀 있는 경우, 시력에 영향을 줄 수 있으므로 즉시 시술을 받는 것이 좋습니다. 시술 전에 눈을 청결하게 하고, 시술 후에는 눈을 보호하기 위해 안경을 착용하는 것이 좋습니다. 또한, 시술 후에는 시력이 회복되는 데 시간이 걸릴 수 있으므로, 시술 후에 시력이 회복되는 데 시간
Q4: True: 그러면 통증이 발생한 곳 이외에 다른 곳에서도 느껴지는 건가요?, / 
Q5: True: 별 말씀을요. 언제든지 도움이 필요하시면 언제든지 저에게 연락해주세요. 쾌차하시길 바랍니다., / 
Q6: True: 아기에게는 다량의 액체 섭취와 식이 조절이 중요합니다. 섭취하는 음식은 가벼운 식이로 유지하고, 소금, 설탕, 유제품 등을 제한해야 합니다. 또한, 청량음료나 커피, 알코올은 피해야 합니다. 설사 증상이 계속되면 다시 진료를 받아야 합니다., /  아기의 설사와 녹색 변은 소화력이 약하거나 흡수가 원활하지 않아서 또는 장 운동이 빠르기 때문입니다. 치료를 위해 항생제인 오플록스-OZ를 식전 1시간 전에 빈 속으로 

In [20]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [100]:
y_pred_real = corrected_outputs

In [121]:
ffff = [i[len(j):].strip() for i,j in zip(y_pred_real,question)]

# Performance

In [122]:

# 성능 평가 계산

def exact_match_score(y_true_real, ffff):
    return sum([1 if pred.lower() == true.lower() else 0 for pred, true in zip(ffff, y_true_real)]) / len(y_true_real)

exact_match = exact_match_score(y_true_real, ffff)

# Hugging Face 토크나이저로 토큰화
def tokenize_text(text):
    return tokenizer.tokenize(text)

bleu_scores = [sentence_bleu([tokenize_text(true)], tokenize_text(pred)) for true, pred in zip(y_true_real, ffff)]
avg_bleu = sum(bleu_scores) / len(bleu_scores)

rouge = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_scores = [rouge.score(true, pred) for true, pred in zip(y_true_real, ffff)]
avg_rouge1 = sum([score["rouge1"].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score["rouge2"].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score["rougeL"].fmeasure  for score in rouge_scores]) / len(rouge_scores)

# METEOR Score 수정 (Hugging Face 토크나이저 사용)
meteor_scores = [meteor_score([tokenize_text(true)], tokenize_text(pred)) for true, pred in zip(y_true_real, ffff)]
avg_meteor = sum(meteor_scores) / len(meteor_scores)

# 결과 출력
print("Exact Match Score:", exact_match)
print("BLEU Score:", avg_bleu)
print("ROUGE-1 Score:", avg_rouge1)
print("ROUGE-2 Score:", avg_rouge2)
print("ROUGE-L Score:", avg_rougeL)
print("METEOR Score:", avg_meteor)    

/home/woogoonkyu/anaconda3/envs/rapa_py311/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/woogoonkyu/anaconda3/envs/rapa_py311/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/woogoonkyu/anaconda3/envs/rapa_py311/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower o

Exact Match Score: 0.0
BLEU Score: 0.022550843751114828
ROUGE-1 Score: 0.01606128475858661
ROUGE-2 Score: 0.0031518509596081268
ROUGE-L Score: 0.01570661787778145
METEOR Score: 0.13941742174293847


In [123]:
pd.DataFrame([avg_bleu, avg_rouge1, avg_rouge2, avg_rougeL, avg_meteor]).T

,0,1,2,3,4
0,0.022551,0.016061,0.003152,0.015707,0.139417


In [125]:
from bert_score import score
import torch

# BERTScore 계산
P, R, F1 = score(
    ffff, y_true_real,
    model_type='klue/bert-base',
    num_layers=12,
    lang='ko',
    verbose=True,
    device="cuda" if torch.cuda.is_available() else "cpu"
)


calculating scores...
computing bert embedding.


100%|██████████| 70/70 [00:08<00:00,  8.40it/s]


computing greedy matching.


100%|██████████| 38/38 [00:00<00:00, 125.13it/s]


done in 8.78 seconds, 270.24 sentences/sec


In [113]:
print(y_pred_real[-5]), print(y_true_real[-5])

You are a medical professional proficient in both Korean and English.
Your task is to carefully read the conversation between a Client and a Doctor, 
then predict the next response from the Doctor in a professional and empathetic manner.

Please respond in **Korean** only.

Now produce the next Doctor response to the Client’s last question.
Return your answer in Korean.

You will always respond as the doctor, and your answer must be written in Korean only.

***Conversation:
갑상선 기능 검사를 해보는 게 좋을 것 같아요. TSH, free T4, T3 수치를 측정해야 할 것 같아요.
그렇게 하겠습니다. 그리고 Graves병을 확인하기 위해 갑상선 자극 항체(TSI) 검사도 해보는 것이 좋을 것 같습니다. 어떻게 대처해야 할지 궁금하신가요?
네, 어떻게 대처해야 할지 알고 싶어요.

***Answer: (의사 답변)  TSH 수치가 정상 범위보다 높게 나왔습니다. 이 경우, 갑상선 기능 저하증을 의심할 수 있습니다. T4, T3 수치를 측정하여 갑상선 기능을 확인할 수 있습니다. 또한, Graves병을 확인하기 위해 갑상선 자극 항체(TSI) 검사를 추천합니다. TSI 검사 결과에 따라 적절한 대처 방안을 마련할 수 있습니다. TSI 검사 결과를 기다리시면 됩니다. *** (의사 답변)  (번역: TSH
Graves병의 경우, 치료 옵션으로 항갑상선 약물, 방사성 요오드 치료, 수술이 있습니다. 어떤 치료 방법이 가장 적합한지는 검사 결과와 상담을 통해 결정해야 합니다. 추가적인 검사와 상담

(None, None)

In [126]:

print(f'BERT_score : {F1.mean()}')

BERT_score : 0.43115168809890747


In [25]:
model_name

'/nas_homes/seunguk/hf_llm/models--Qwen--Qwen2.5-7B/snapshots/d149729398750b98c0af14eb82c78cfe92750796'